#

$$y=\frac{\max}{1+e^{k(T-T_0)}}+\min$$

In [97]:
import scipy.optimize
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact


data = {
    10: (
        [100, 150, 200, 220, 230, 250, 300, 350, 400],
        [283, 267, 249, 232, 221, 187, 97,   45,  34],
        [254.0, 31.0, 0.03, 260.0]        
    ),
    20: (
        [100, 150, 200, 220, 230, 250, 300, 350, 400],
        [283, 267, 226, 208, 191,  81,  46,  42,  34],
        [254.0, 31.0, 0.05, 230.0]        
    ),
    30: (
        [100, 150, 200, 220, 230, 250, 300, 350, 400],
        [285, 267, 226, 168,  93,  74,  44,  38,  34],
        [254.0, 31.0, 0.05, 220.0]        
    ),

}


def error(xs, ys):
    """Вычисляем максимальную относительную ошибку"""
    series = [np.abs(x-y)/x for x, y in zip(xs, ys)]
    return np.max(series)

def f(params):
    max, min, k, t0 = params
    y = lambda t: max/(1+np.exp(k*(t-t0)))+min
    return np.frompyfunc(y, 1, 1)

def output(params):
    print(f'               max     min       k       t0   δ')
    for param in params:
        title, (max, min, k, t0), err = param
        print(f'{title}  {max:.3f} {min:.3f} {k:.5f} {t0:.3f} {err:.2f}%')



In [102]:
def show(minutes):
    ts, cs, x0 = data[minutes]

    def target_sqr(params):
        max, min, k, t0 = params
        sum = 0
        for t, c in zip(ts, cs):
            sum += (max/(1+np.exp(k*(t-t0)))+min-c)**2
        return sum

    def target_abs(params):
        max, min, k, t0 = params
        sum = 0
        for t, c in zip(ts, cs):
            sum += np.abs(max/(1+np.exp(k*(t-t0)))+min-c)
        return sum

    res_sqr = scipy.optimize.minimize(target_sqr, x0=x0, method='BFGS')
    res_abs = scipy.optimize.minimize(target_abs, x0=x0, method='BFGS')
    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(ts, cs, marker='o', color = 'blue', label='Эксперимент')

    xs = np.linspace(100, 400, 100)
    y = f(x0)
    err_vis = error(cs, y(xs))
    ax.plot(xs, y(xs), color = 'red', label= 'Визуальный подбор')
    y = f(res_sqr.x)
    err_sqr = error(cs, y(xs))
    ax.plot(xs, y(xs), color = 'green', label='sqr -> min (BFGS)')
    y = f(res_abs.x)
    err_abs = error(cs, y(xs))
    ax.plot(xs, y(xs), color = 'purple', label='abs -> min (BFGS)')
    ax.legend();

    output(zip(
        ['Визуально: ', 'sqr -> min:', 'abs -> min:'],
        [x0, res_sqr.x, res_abs.x],
        [err_vis, err_sqr, err_abs],
    ))

#show(10)


In [103]:
interact(show, minutes=[10, 20, 30]);


interactive(children=(Dropdown(description='minutes', options=(10, 20, 30), value=10), Output()), _dom_classes…